# This file contains a test script that checks the update script Tanzil Wrote

In [50]:
import pandas as pd
import requests
import re
from fuzzywuzzy import process

In [51]:
ideal_promises = pd.read_excel('../assets/data/Promises.xlsx')

In [59]:
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQi5APAZW3CvMw-8LUdCFxdg5CIEjAB-EnnKHe6kzMxQyHDWZbZToW3kLSvpDMcEQZ34o6nOIxbMcwm/pub?output=xlsx"

In [64]:
googleFormUpdates = requests.get(url)

In [66]:
with open('../assets/data/googleFormUpdates.xlsx', 'wb') as f:
    f.write(googleFormUpdates.content)

In [68]:
updates = pd.read_excel('../assets/data/googleFormUpdates.xlsx').rename(columns={'Promise ': 'Promise', 'SubPromise ': 'Sub promise'}).dropna(how='all')

In [71]:
for col in ['Promise', 'Sub promise']:
    ideal_promises[col] = ideal_promises[col].apply(lambda x: x.lower().strip())
    updates[col] = updates[col].apply(lambda x: x.lower().strip())

In [72]:
correct_subpromises = set(ideal_promises['Sub promise'].unique())
correct_promises = set(ideal_promises['Promise'].unique())
updates_promises = set(updates['Promise'].unique())

In [73]:
errors = updates_promises - correct_promises
corrections = {k:process.extract(k, correct_promises, limit=1)[0][0] for k in errors}
maps = {k:k for k in updates['Promise'].unique()}
maps.update(corrections)
updates['Promise'] = updates['Promise'].map(maps)
updates['Sub promise'] = updates['Sub promise'].apply(lambda x: x.split('__')[-1].lower().strip())
updates[updates['Sub promise'].isin(correct_subpromises)].to_excel('../assets/data/googleFormUpdates.xlsx', index=None)

In [77]:
oldupdates = pd.read_excel('../assets/data/old_website_updates.xlsx')

In [83]:
combined = pd.concat([updates, oldupdates], sort=False)

In [84]:
merged_df = pd.merge(combined, ideal_promises, left_on='Sub promise', right_on = 'Sub promise')

In [88]:
merged_df = merged_df.drop(['Department_x', 'Promise_x', 'Outcome_x', 'Sr No'], axis=1).rename(columns={'Promise_y':'Promise', 'Department_y':'Department', 'Outcome_y':'Outcome'})

In [94]:
merged_df['Department'] = merged_df['Department'].apply(lambda x: re.sub('\sDepartment', '', x))

In [104]:
merged_df.to_excel('../assets/data/updates.xlsx', index=None)